In [40]:
from operator import attrgetter
from collections import defaultdict
import math
import numpy as np
import pandas as pd
from collections import namedtuple

In [41]:
# Algorithm
Dota2Metrics = namedtuple(
    'Dota2Metrics',
    ['part_number', 'has_winned', 'kills', 'assists', 'deaths', 'gold_per_minute', 'hero_damage']
)

def calc_score_dota2(v: Dota2Metrics):
    return (
        v.kills * 1.0 +
        v.assists * 0.5
        - v.deaths * 0.6 +
        v.gold_per_minute * 0.001 +
        v.hero_damage / 10000.0
    )


def calc_score_cs2(v):
    ...


def calc_score_lol(v):
    ...


def calc_total_score(m: Dota2Metrics):
    MATCH_PART_FACTOR = 10.0
    WIN_LOSE_FACTOR = np.array([0.5, 1.0], np.float32)

    s = np.array(list(map(calc_score_dota2, m)), np.float32)

    part_ns = np.array(
        list(map(attrgetter("part_number"), m)),
        np.float32
    )
    np.add(part_ns, 1.0, out=part_ns)
    np.multiply(part_ns, MATCH_PART_FACTOR, out=part_ns)

    wins = np.array(
        list(map(attrgetter("has_winned"), m)),
        np.int32
    )

    np.multiply(s, np.log10(part_ns) / math.log10(MATCH_PART_FACTOR), out=s)
    np.multiply(s, np.take(WIN_LOSE_FACTOR, wins), out=s)
    return np.sum(s)


In [42]:
# Dataset
teams_boom = {
    "Pakazs": [ Dota2Metrics(0, 0, 8, 10, 1, 710, 20577) ],
    "SLATEM$": [ Dota2Metrics(0, 0, 10, 9, 1, 706, 20927) ],
    "Sacred": [ Dota2Metrics(0, 0, 8, 9, 2, 647, 19845) ],
    "Matthew": [ Dota2Metrics(0, 0, 5, 14, 2, 381, 10670) ],
    "Mjz": [ Dota2Metrics(0, 0, 1, 18, 1, 297, 9065) ]
}

teams_mad_kings = {
    "Adrian": [
        Dota2Metrics(0, 1, 0, 3, 5, 582, 16522),
        Dota2Metrics(1, 0, 12, 7, 4, 718, 30968)
    ],
    "Mr.Jeans": [
        Dota2Metrics(0, 1, 3, 2, 5, 531, 15305),
        Dota2Metrics(1, 0, 7, 10, 5, 597, 25978)    
    ],
    "Smash": [
        Dota2Metrics(0, 1, 0, 1, 5, 441, 12945),
        Dota2Metrics(1, 0, 2, 10, 7, 437, 18744)
    ],
    "Genek": [
        Dota2Metrics(0, 1, 3, 2, 10, 247, 5573),
        Dota2Metrics(1, 0, 0, 15, 4, 275, 11037)],
    "Yadomi": [
        Dota2Metrics(0, 1, 1, 3, 7, 199, 4254),
        Dota2Metrics(1, 0, 1, 11, 7, 288, 10908)
    ]
}

team_boca_juniors = {
    "Parker": [ Dota2Metrics(0, 0, 3, 2, 3, 579, 20395) ],
    "Latotronii": [ Dota2Metrics(0, 0, 2, 4, 4, 477, 17546) ],
    "Italiano Gangstar": [ Dota2Metrics(0, 0, 4, 4, 4, 508, 18106) ],
    "Mariano": [ Dota2Metrics(0, 0, 2, 4, 4, 303, 12146) ],
    "Accel": [ Dota2Metrics(0, 0, 0, 7, 12, 241, 6556) ]
}

team_infinity = {
    "payk": [
        Dota2Metrics(0, 1, 7, 12, 2, 720, 28335),
        Dota2Metrics(1, 1, 14, 9, 0, 816, 47479)
    ],
    "Lumpy": [
        Dota2Metrics(0, 1, 7, 18, 0, 613, 22975),
        Dota2Metrics(1, 1, 5, 12, 4, 579, 28811)
    ],
    "Vitaly": [
        Dota2Metrics(0, 1, 7, 12, 0, 673, 25862),
        Dota2Metrics(1, 1, 4, 17, 2, 775, 38214)
    ],
    "MoOz": [
        Dota2Metrics(0, 1, 4, 16, 2, 375, 12601),
        Dota2Metrics(1, 1, 2, 17, 8, 310, 14417)
    ],
    "Gardick": [
        Dota2Metrics(0, 1, 1, 16, 7, 345, 10185),
        Dota2Metrics(1, 1, 2, 15, 8, 342, 15646)
    ]
}

teams = {
    "BOOM Esports": teams_boom,
    "Mad Kings": teams_mad_kings,
    "BocaJuniors": team_boca_juniors,
    "Infinity": team_infinity
}

In [43]:
# Searching results
player_scores = {}
player_teams = {}
teams_scores = defaultdict(lambda: 0.0)

for team_name in teams:
    team = teams[team_name]
    for player_name in team:
        player_scores[player_name] = calc_total_score(team[player_name])
        player_teams[player_name] = team_name
        teams_scores[team_name] += player_scores[player_name]
    teams_scores[team_name] /= len(team)


# Calculate players table
players_df_schema = {
    'Команда': 'str',
    'Игрок': 'str',
    'Счет': 'float32'
}
players_df = pd.DataFrame(columns=players_df_schema.keys()).astype(players_df_schema)
for i, p_name in enumerate(player_scores):
    players_df.loc[i] = (player_teams[p_name], p_name, player_scores[p_name])

players_df = players_df.sort_values('Счет', ascending=False)

display(players_df)


# Calculate teams table
teams_df_schema = {
    'Команда': 'str',
    'Ср. счет': 'float32'
}
teams_df = pd.DataFrame(columns=teams_df_schema.keys()).astype(teams_df_schema)
for i, t_name in enumerate(teams_scores):
    teams_df.loc[i] = (t_name, teams_scores[t_name])

teams_df = teams_df.sort_values('Ср. счет', ascending=False)

display(teams_df)

,Команда,Игрок,Счет
15,Infinity,payk,46.661358
17,Infinity,Vitaly,36.940895
16,Infinity,Lumpy,34.601051
18,Infinity,MoOz,22.129986
19,Infinity,Gardick,14.758884
5,Mad Kings,Adrian,11.737532
6,Mad Kings,Mr.Jeans,10.994401
1,BOOM Esports,SLATEM$,8.349350
0,BOOM Esports,Pakazs,7.583850
2,BOOM Esports,Sacred,6.965750


,Команда,Ср. счет
3,Infinity,31.018435
0,BOOM Esports,6.864940
1,Mad Kings,5.925756
2,BocaJuniors,1.488290
